In [1]:
# The FB flight for the $5 Off coupon was 12/5-12/15
# From Margaret, RE: BL $5 Reward December analysis request (FB+EM vs. EM only)
# Tuesday, January 14, 2020 at 3:20 PM

import pandas as pd
import os
import datetime
import numpy as np
import gc
import glob
gc.collect()

datetime.datetime.now()

datetime.datetime(2020, 3, 13, 17, 37, 42, 700857)

In [2]:
df_redeemers=pd.read_csv("/home/jian/BigLots/AgilOne/redeemers_2019Dec/december_full_file_redeemers.txt",
                        dtype=str,sep="\t")
print(df_redeemers.shape)
print(df_redeemers.shape,df_redeemers['hashed_customer_num'].nunique(),df_redeemers['hashed_email_address'].nunique())
df_redeemers.head(2)

(249159, 2)
(249159, 2) 249159 249159


,hashed_customer_num,hashed_email_address
0,e636946300d822417f695bd93a14a2bc6506ca3be6485c...,35b15f9589fa9ed6edbf377fbdeacd201eadfab15c118d...
1,5c3890970390e6fc48245643ea27e03c584f0cbf957afd...,460d11ff27714f2ef27b05cc1164466d13ced6af048bd3...


In [3]:
df_FB_Control=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Facebook_coupons/hash_facebook_only_1201.zip",
                             dtype=str,compression="zip",sep="|")
df_FB_Test=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Facebook_coupons/hash_facebook_email_1201.zip",
                             dtype=str,compression="zip",sep="|")

print("df_FB_Control.shape",df_FB_Control.shape)
print("df_FB_Test.shape",df_FB_Test.shape)


df_FB_Control.shape (500000, 2)
df_FB_Test.shape (6400000, 2)


In [4]:
df_FB_Test['Group']="test"
df_FB_Control['Group']="control"
df_id_group=df_FB_Test.append(df_FB_Control)

del df_FB_Test
del df_FB_Control
print(df_id_group.shape)

df_id_group.head(2)

(6900000, 3)


,hashed_customer_num,hashed_email_address,Group
0,913adf1bb21c381d129a226468e69ce8a61c07083efbd5...,7c872f1fd8985a0cee7e6d8b86fa89f34c683d018111ab...,test
1,dad71dc369858e4d71755b3b88701a3dea4de62eef8ac7...,28bb15e7436c23796ebf07a42e3787bfabf1ed8ad9eb3a...,test


In [5]:
df_qc_duplicate_id=df_id_group.groupby("hashed_customer_num")['Group'].nunique().to_frame().reset_index().sort_values("Group",ascending=False)
df_qc_duplicate_email=df_id_group.groupby("hashed_email_address")['Group'].nunique().to_frame().reset_index().sort_values("Group",ascending=False)

df_qc_duplicate_id=df_qc_duplicate_id[df_qc_duplicate_id['Group']>1]
print(df_qc_duplicate_id.shape)

df_qc_duplicate_email=df_qc_duplicate_email[df_qc_duplicate_email['Group']>1]
print(df_qc_duplicate_email.shape)


(0, 2)
(12, 2)


In [6]:
list_duplicated_emails=df_qc_duplicate_email['hashed_email_address'].tolist()
print(len(list_duplicated_emails))

list_duplicated_ids=df_id_group[df_id_group['hashed_email_address'].isin(list_duplicated_emails)]
print(len(list_duplicated_ids))


12
24


In [7]:
# 12 emails removed --24 ids
df_id_group=df_id_group[~df_id_group['hashed_email_address'].isin(list_duplicated_emails)]
del df_id_group['hashed_email_address']
df_id_group=df_id_group.rename(columns={"hashed_customer_num":"customer_id_hashed"})
print(df_id_group.shape)

(6899976, 2)


# Clickers

In [8]:
'''
import paramiko

host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

remote_path="/mnt/drv5/jian/Big_Lots/BL_1_to_1_2019Q4/Aggregated_files_to_use/LR/BL_aggregate_click_LR_returned_Q4_JL_2020-02-19.tsv.gz"
base_name=os.path.basename(remote_path)
local_path=os.getcwd()+"/"+base_name

print(local_path)


sftp.get(remote_path,local_path)
sftp.close()
transport.close()
'''

'\nimport paramiko\n\nhost = "64.237.51.251" #hard-coded\nport = 22\ntransport = paramiko.Transport((host, port))\n\npassword = "jian@juba2017" #hard-coded\nusername = "jian" #hard-coded\ntransport.connect(username = username, password = password)\nsftp = paramiko.SFTPClient.from_transport(transport)\n\nremote_path="/mnt/drv5/jian/Big_Lots/BL_1_to_1_2019Q4/Aggregated_files_to_use/LR/BL_aggregate_click_LR_returned_Q4_JL_2020-02-19.tsv.gz"\nbase_name=os.path.basename(remote_path)\nlocal_path=os.getcwd()+"/"+base_name\n\nprint(local_path)\n\n\nsftp.get(remote_path,local_path)\nsftp.close()\ntransport.close()\n'

In [9]:
df_clickers=pd.read_csv("./BL_aggregate_click_LR_returned_Q4_JL_2020-02-19.tsv.gz",
                       compression="gzip",sep="\t",dtype=str,
                        usecols=['Customer_Link','Event Time','Advertiser ID','Campaign ID','Ad ID','Site ID (DCM)','Placement ID'])
print(df_clickers.shape)
df_clickers=df_clickers[df_clickers['Customer_Link']!="UNMATCHED"]
print(df_clickers.shape)
df_clickers.shape,df_clickers['Customer_Link'].nunique()

(1600823, 7)
(296107, 7)


((296107, 7), 227224)

In [10]:
list_placement_id=["262271523","262377066","262281871","262377075","262271526","262377072","262271301",
                   "262377990","261974462","262281874","262179760","262377063"]

df_clickers=df_clickers[df_clickers['Placement ID'].isin(list_placement_id)]

df_clickers.shape,df_clickers['Customer_Link'].nunique()

((27077, 7), 20713)

In [11]:
df_clickers['datetime']=df_clickers['Event Time'].astype(int)
df_clickers['datetime']=pd.to_datetime(df_clickers['datetime'],unit="us")
df_clickers.head(2)

,Customer_Link,Event Time,Advertiser ID,Campaign ID,Ad ID,Site ID (DCM),Placement ID,datetime
753890,XY1468UpBX2PWxX32z0epWdX-hsC-NG_Ol0xUi6q7iaREUJmQ,1575559993545088,8095847,23407120,457907791,1291657,262377075,2019-12-05 15:33:13.545088
753894,XY1468GyEZbawTirlwnZmX9p8MJJrKDBl8XRXbMZ-nqTTPfmE,1575558805837440,8095847,23407120,457817072,1291657,262377066,2019-12-05 15:13:25.837440


In [14]:
df_clickers['date']=df_clickers['datetime'].dt.date
df_qc_by_date=df_clickers.groupby("date")['Event Time'].count().to_frame().reset_index()
df_qc_by_date.sort_values("date")

,date,Event Time
0,2019-12-05,2158
1,2019-12-06,3235
2,2019-12-07,3590
3,2019-12-08,3198
4,2019-12-09,2951
5,2019-12-10,2397
6,2019-12-11,2118
7,2019-12-12,1685
8,2019-12-13,1835
9,2019-12-14,1737


In [15]:
df_mapping=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/CL_BigLots_BL_mapping_file_2019Q4_JL_2020-02-14_20200215_020719_0000.psv.gz",
                      compression="gzip",sep="|",dtype=str)
df_mapping.shape

(25087634, 3)

In [16]:
df_clickers=pd.merge(df_clickers,df_mapping,on="Customer_Link",how="left")
print(df_clickers.shape)

list_customer_ids_clickers=df_clickers['customer_id_hashed'].unique().tolist()
print(1,len(list_customer_ids_clickers))

df_click_ids=pd.DataFrame({"customer_id_hashed":list_customer_ids_clickers})
print(2,df_click_ids.shape)

df_click_ids=df_click_ids[~pd.isnull(df_click_ids['customer_id_hashed'])]
print(3,df_click_ids.shape)

df_click_ids=df_click_ids[~df_click_ids['customer_id_hashed'].isin(list_duplicated_ids)]
print(4,df_click_ids.shape)


(27268, 11)
1 3660
2 (3660, 1)
3 (3659, 1)
4 (3659, 1)


In [17]:
df_id_group.head(2)

,customer_id_hashed,Group
0,913adf1bb21c381d129a226468e69ce8a61c07083efbd5...,test
1,dad71dc369858e4d71755b3b88701a3dea4de62eef8ac7...,test


In [18]:
df_click_ids.head(2)

,customer_id_hashed
1,a50a2c0ec9e6b6bb9454bd053fd07eed6956fbb8fae003...
2,ab3efd98d906269ab7ad309166f58c844f597ca07aa5c8...


In [19]:
df_redeemers.head(2)

,hashed_customer_num,hashed_email_address
0,e636946300d822417f695bd93a14a2bc6506ca3be6485c...,35b15f9589fa9ed6edbf377fbdeacd201eadfab15c118d...
1,5c3890970390e6fc48245643ea27e03c584f0cbf957afd...,460d11ff27714f2ef27b05cc1164466d13ced6af048bd3...


In [20]:
df_FB_id_all=df_id_group.append(df_click_ids).drop_duplicates("customer_id_hashed",keep="first")
df_FB_id_all['Group']=df_FB_id_all['Group'].fillna("clicker_not_uploaded")

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [21]:
df_FB_id_all.head(2)

,Group,customer_id_hashed
0,test,913adf1bb21c381d129a226468e69ce8a61c07083efbd5...
1,test,dad71dc369858e4d71755b3b88701a3dea4de62eef8ac7...


In [22]:
df_redeemers_others=df_redeemers[~df_redeemers['hashed_customer_num'].isin(df_FB_id_all['customer_id_hashed'].tolist())]
# df_redeemers_others=df_redeemers_others[~df_redeemers_others['hashed_email_address'].isin(df_FB_id_email_all['email_address_hash'].tolist())]
print(df_redeemers_others.shape)
df_redeemers_others['Group']="redeemers_not_uploaded"
df_redeemers_others=df_redeemers_others[['hashed_customer_num','Group']].rename(columns={"hashed_customer_num":"customer_id_hashed"})
df_FB_id_all=df_FB_id_all.append(df_redeemers_others)

(164584, 2)


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
df_FB_id_all.head(2)

,Group,customer_id_hashed
0,test,913adf1bb21c381d129a226468e69ce8a61c07083efbd5...
1,test,dad71dc369858e4d71755b3b88701a3dea4de62eef8ac7...


In [24]:
df_FB_id_all.groupby("Group")['customer_id_hashed'].count().to_frame().reset_index()

,Group,customer_id_hashed
0,clicker_not_uploaded,1118
1,control,499988
2,redeemers_not_uploaded,164584
3,test,6399988


In [25]:
list_redeamer_id=df_redeemers['hashed_customer_num'].tolist()
df_FB_id_all['redemption']=np.where(df_FB_id_all['customer_id_hashed'].isin(list_redeamer_id),"redeemers","others" )
df_FB_id_all['FB_clicker']=np.where(df_FB_id_all['customer_id_hashed'].isin(list_customer_ids_clickers),"clickers","non_clickers")


In [26]:
df_unsub_emails=pd.read_csv("/home/jian/BigLots/unsubscribe/Unsub_JH_Final20191105Hashed.zip",
                              compression="zip",dtype=str)
list_unsub_emails=df_unsub_emails['hashed_email_address'].unique().tolist()
list_unsub_ids=df_unsub_emails['hashed_customer_num'].unique().tolist()

print(df_unsub_emails.shape,len(list_unsub_ids),len(list_unsub_emails))

df_FB_id_all['unsubscription']=np.where(df_FB_id_all['customer_id_hashed'].isin(list_unsub_ids),"unsubscribed","default_subscribed")


(2771375, 2) 2771375 2771350


# Sales

In [27]:
# POS data range
file_week_1="/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-07/MediaStormDailySales20191211.txt"
file_week_2="/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-14/MediaStormDailySales20191217-195625-000.txt"
file_week_3="/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-21/MediaStormDailySales20191226-122746-000.txt"

POS_file_list=[file_week_1,file_week_2,file_week_3]

df_POS_all=pd.DataFrame()

for file in POS_file_list:
    df=pd.read_table(file,dtype=str,sep="|")
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['sales_instore']=np.where(df['location_id']!="6990",df['item_transaction_amt'],0)
    df['sales_online']=np.where(df['location_id']=="6990",df['item_transaction_amt'],0)
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans['trans_instore']=np.where(df_trans['location_id']!="6990",1,0)
    df_trans['trans_online']=np.where(df_trans['location_id']=="6990",1,0)
    
    df_trans=df_trans.groupby(["customer_id_hashed","transaction_dt"])['trans_instore','trans_online'].sum().reset_index()
    
    df_sales=df.groupby(["customer_id_hashed","transaction_dt"])['sales_instore','sales_online'].sum().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed","transaction_dt"],how="outer")
    df_POS_all=df_POS_all.append(df)
    print(datetime.datetime.now(),df_POS_all.shape)
    
df_POS_all.shape

2020-03-13 17:52:57.243627 (2202929, 6)
2020-03-13 17:55:16.206657 (4393434, 6)
2020-03-13 17:57:39.717374 (6671816, 6)


(6671816, 6)

In [28]:
P1_range=["2019-12-05","2019-12-15"]
P2_range=["2019-12-16","2019-12-20"]

In [29]:
print(df_POS_all['transaction_dt'].min(),df_POS_all['transaction_dt'].max())

2019-12-01 2019-12-21


In [30]:
print(df_POS_all['transaction_dt'].min(),df_POS_all['transaction_dt'].max())
print(df_FB_id_all.shape)
df_FB_id_all.head(2)

2019-12-01 2019-12-21
(7065678, 5)


,Group,customer_id_hashed,redemption,FB_clicker,unsubscription
0,test,913adf1bb21c381d129a226468e69ce8a61c07083efbd5...,others,non_clickers,default_subscribed
1,test,dad71dc369858e4d71755b3b88701a3dea4de62eef8ac7...,others,non_clickers,default_subscribed


In [31]:
df_FB_id_all.shape, df_FB_id_all['customer_id_hashed'].nunique()

((7065678, 5), 7065678)

In [32]:
# left merge into the POS
df_POS_all=pd.merge(df_POS_all,df_FB_id_all,on="customer_id_hashed",how="left")
print(df_POS_all.shape)
df_POS_all.head(2)

(6671816, 10)


,customer_id_hashed,transaction_dt,sales_instore,sales_online,trans_instore,trans_online,Group,redemption,FB_clicker,unsubscription
0,00000435bfb3bf42e3beb4c9b3942c552d09f0e49e5a75...,2019-12-07,18.0,0.0,1,0,redeemers_not_uploaded,redeemers,non_clickers,default_subscribed
1,000008374518306e87838f387eedc47d49a447707dc6da...,2019-12-03,55.9,0.0,1,0,redeemers_not_uploaded,redeemers,non_clickers,default_subscribed


In [33]:
df_POS_all['Period']=np.where(((df_POS_all['transaction_dt']>=P1_range[0]) & (df_POS_all['transaction_dt']<=P1_range[1])),"P1: 12/05-12/15",
                              np.where(((df_POS_all['transaction_dt']>=P2_range[0]) & (df_POS_all['transaction_dt']<=P2_range[1])),"P2: 12/16-12/20","Others")
                             )

In [34]:
df_POS_all.to_csv("./BL_df_detail_3_weeks_POS_instore_and_online_with_redeemers_subscribe_FBclicker_JL_"+str(datetime.datetime.now().date())+".csv",index="False")


In [35]:
agg_func={"customer_id_hashed":"nunique","sales_instore":"sum","sales_online":"sum","trans_instore":"sum","trans_online":"sum"}
df_summary=df_POS_all.groupby(["Group","Period","redemption","unsubscription","FB_clicker"]).agg(agg_func).reset_index()
df_summary.shape

(54, 10)

In [36]:
df_total_id=df_FB_id_all.groupby(["Group","redemption","unsubscription","FB_clicker"])['customer_id_hashed'].nunique().to_frame().reset_index()
df_total_id

,Group,redemption,unsubscription,FB_clicker,customer_id_hashed
0,clicker_not_uploaded,others,default_subscribed,clickers,983
1,clicker_not_uploaded,others,unsubscribed,clickers,115
2,clicker_not_uploaded,redeemers,default_subscribed,clickers,20
3,control,others,default_subscribed,clickers,200
4,control,others,default_subscribed,non_clickers,496789
5,control,others,unsubscribed,clickers,1
6,control,others,unsubscribed,non_clickers,243
7,control,redeemers,default_subscribed,clickers,16
8,control,redeemers,default_subscribed,non_clickers,2726
9,control,redeemers,unsubscribed,non_clickers,13


In [37]:
df_unique_shoppers_both_periods=df_POS_all[df_POS_all['Period']!="Others"]
df_unique_shoppers_both_periods=df_unique_shoppers_both_periods.groupby(["Group","redemption","unsubscription","FB_clicker"])['customer_id_hashed'].nunique().to_frame().reset_index()
df_unique_shoppers_both_periods.shape

(18, 5)

In [38]:
writer=pd.ExcelWriter("./BL_output_FB_Dec_offer_result_by_store_type_redemption_unsubscription_clickers_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary.to_excel(writer,"performance",index=False)
df_total_id.to_excel(writer,"total_id",index=False)
df_unique_shoppers_both_periods.to_excel(writer,"total_shoppers_both_P",index=False)
writer.save()

In [39]:
df_summary.head(2)

,Group,Period,redemption,unsubscription,FB_clicker,customer_id_hashed,sales_instore,sales_online,trans_instore,trans_online
0,clicker_not_uploaded,Others,others,default_subscribed,clickers,46,1892.22,0.0,52,0
1,clicker_not_uploaded,Others,others,unsubscribed,clickers,3,101.93,0.0,4,0
